# Convert XML into a single CSV file

In [3]:
import json
import os
import glob
import pandas as pd
import argparse
import xml.etree.ElementTree as ET


def __list_to_csv(annotations, output_file):
    column_name = [
        'filename', 'width', 'height', 'class', 'xmin', 'ymin', 'xmax', 'ymax'
    ]
    xml_df = pd.DataFrame(annotations, columns=column_name)
    xml_df.to_csv(output_file, index=None)


def xml_to_csv(xml_dir, output_file):
    """Reads all XML files, generated by labelImg, from a directory and generates a single CSV file"""
    annotations = []
    for xml_file in glob.glob(xml_dir + '/*.xml'):
        tree = ET.parse(xml_file)
        root = tree.getroot()
        for member in root.findall('object'):
            value = (root.find('filename').text,
                     int(root.find('size')[0].text),
                     int(root.find('size')[1].text), member[0].text,
                     int(member[4][0].text), int(member[4][1].text),
                     int(member[4][2].text), int(member[4][3].text))
            annotations.append(value)

    __list_to_csv(annotations, output_file)


def json_to_csv(input_json, output_file):
    """Reads a JSON file, generated by the VGG Image Annotator, and generates a single CSV file"""
    with open(input_json) as f:
        images = json.load(f)

    annotations = []

    for entry in images:
        filename = images[entry]['filename']
        for region in images[entry]['regions']:
            c = region['region_attributes']['class']
            xmin = region['shape_attributes']['x']
            ymin = region['shape_attributes']['y']
            xmax = xmin + region['shape_attributes']['width']
            ymax = ymin + region['shape_attributes']['height']
            width = 0
            height = 0

            value = (filename, width, height, c, xmin, ymin, xmax, ymax)
            annotations.append(value)

    __list_to_csv(annotations, output_file)


if __name__ == "__main__":
    parser = argparse.ArgumentParser(
        description=
        'Reads all XML files, generated by labelImg, from a directory and generates a single CSV file',
        formatter_class=argparse.RawDescriptionHelpFormatter)
    parser.add_argument('type',
                        metavar='type',
                        default='xml',
                        choices=['xml', 'json'],
                        help='LabelImg XML or VIA JSON')
    parser.add_argument(
        'input',
        metavar='input',
        type=str,
        help=
        'Directory containing the XML files generated by labelImg or path to a single VIA JSON'
    )
    parser.add_argument('output_csv',
                        metavar='output_csv',
                        type=str,
                        help='Path where the CSV output will be created')

    args = parser.parse_args()

    if args.type == 'xml':
        xml_to_csv(args.input, args.output_csv)
    elif args.type == 'json':
        json_to_csv(args.input, args.output_csv)

usage: ipykernel_launcher.py [-h] type input output_csv
ipykernel_launcher.py: error: argument type: invalid choice: '/home/eisti/.local/share/jupyter/runtime/kernel-49455a75-14ef-4e6c-b1a8-c6556e6e22ca.json' (choose from 'xml', 'json')
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/lib/python3.6/argparse.py", line 1775, in parse_known_args
    namespace, args = self._parse_known_args(args, namespace)
  File "/usr/lib/python3.6/argparse.py", line 1984, in _parse_known_args
    stop_index = consume_positionals(start_index)
  File "/usr/lib/python3.6/argparse.py", line 1940, in consume_positionals
    take_action(action, args)
  File "/usr/lib/python3.6/argparse.py", line 1833, in take_action
    argument_values = self._get_values(action, argument_strings)
  File "/usr/lib/python3.6/argparse.py", line 2275, in _get_values
    self._check_value(action, value)
  File "/usr/lib/python3.6/argparse.py", line 2327, in _check_value
    raise ArgumentError(action, msg % args)
argparse.ArgumentError: argument type: invalid choice: '/home/eisti/.local/share/jupyter/runtime/kernel-49455a75-14ef-4e6c-b1a8-c6556e6e22ca.json' (choose from 'xml', 'json')

During handling of the above exception, another exception occurred:

Traceback 

SystemExit: 2

/home/eisti/.virtualenvs/ENV_DEEPL/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3339: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


# Separate the CSV file into "train - test"

In [ ]:
import os
import argparse
import pandas as pd
from sklearn.model_selection import train_test_split

if __name__ == "__main__":
    parser = argparse.ArgumentParser(
        description='Separates a CSV file into training and validation sets',
        formatter_class=argparse.RawDescriptionHelpFormatter)
    parser.add_argument(
        'input_csv',
        metavar='input_csv',
        type=str,
        help='Path to the input CSV file')
    parser.add_argument(
        '-f',
        metavar='train_frac',
        type=float,
        default=.75,
        help=
        'fraction of the dataset that will be separated for training (default .75)'
    )
    parser.add_argument(
        '-s',
        metavar='stratify',
        type=bool,
        default=True,
        help='Stratify by class instead of whole dataset (default True)')
    parser.add_argument(
        '-o',
        metavar='output_dir',
        type=str,
        default=None,
        help=
        'Directory to output train and evaluation datasets (default input_csv directory)'
    )

    args = parser.parse_args()

    if args.f < 0 or args.f > 1:
        raise ValueError('train_frac must be between 0 and 1')

    # output_dir = input_csv directory is None
    if args.o is None:
        output_dir, _ = os.path.split(args.input_csv)
    else:
        output_dir = args.o

    df = pd.read_csv(args.input_csv)

    # get 'class' column for stratification
    strat = df['class'] if args.s else None

    train_df, validation_df = train_test_split(
        df, test_size=None, train_size=args.f, stratify=strat)

    # output files have the same name of the input file, with some extra stuff appended
    new_csv_name = os.path.splitext(args.input_csv)[0]
    train_csv_path = os.path.join(output_dir, new_csv_name + '_train.csv')
    eval_csv_path = os.path.join(output_dir, new_csv_name + '_eval.csv')

    train_df.to_csv(train_csv_path, index=False)
    validation_df.to_csv(eval_csv_path, index=False)

# Create label map

In [ ]:
import pandas as pd
import argparse


def pbtxt_from_classlist(l, pbtxt_path):
    pbtxt_text = ''

    for i, c in enumerate(l):
        pbtxt_text += 'item {\n    id: ' + str(
            i + 1) + '\n    display_name: "' + c + '"\n}\n\n'

    with open(pbtxt_path, "w+") as pbtxt_file:
        pbtxt_file.write(pbtxt_text)


def pbtxt_from_csv(csv_path, pbtxt_path):
    class_list = list(pd.read_csv(csv_path)['class'].unique())
    class_list.sort()

    pbtxt_from_classlist(class_list, pbtxt_path)


def pbtxt_from_txt(txt_path, pbtxt_path):
    # read txt into a list, splitting by newlines
    data = [
        l.rstrip('\n').strip()
        for l in open(txt_path, 'r', encoding='utf-8-sig')
    ]

    data = [l for l in data if len(l) > 0]

    pbtxt_from_classlist(data, pbtxt_path)


if __name__ == "__main__":
    parser = argparse.ArgumentParser(
        description=
        'Reads all XML files, generated by labelImg, from a directory and generates a single CSV file')
    parser.add_argument(
        'input_type',
        choices=['csv', 'txt'],
        help=
        'type of input file (csv with at least one \'class\' column or txt with one class name by line)'
    )
    parser.add_argument(
        'input_file',
        metavar='input_file',
        type=str,
        help='Path to the input txt or csv file')
    parser.add_argument(
        'output_file',
        metavar='output_file',
        type=str,
        help='Path where the .pbtxt output will be created')

    args = parser.parse_args()

    if args.input_type == 'csv':
        pbtxt_from_csv(args.input_file, args.output_file)
    elif args.input_type == 'txt':
        pbtxt_from_txt(args.input_file, args.output_file)

# Convert the two CSV files into two TFRecord files

In [ ]:
from __future__ import division
from __future__ import print_function
from __future__ import absolute_import

import os
import io
import pandas as pd
import tensorflow as tf
import argparse

from PIL import Image
from tqdm import tqdm
from object_detection.utils import dataset_util
from collections import namedtuple, OrderedDict


def __split(df, group):
    data = namedtuple('data', ['filename', 'object'])
    gb = df.groupby(group)
    return [
        data(filename, gb.get_group(x))
        for filename, x in zip(gb.groups.keys(), gb.groups)
    ]


def create_tf_example(group, path, class_dict):
    with tf.gfile.GFile(os.path.join(path, '{}'.format(group.filename)),
                        'rb') as fid:
        encoded_jpg = fid.read()
    encoded_jpg_io = io.BytesIO(encoded_jpg)
    image = Image.open(encoded_jpg_io)
    width, height = image.size

    filename = group.filename.encode('utf8')
    image_format = b'jpg'
    xmins = []
    xmaxs = []
    ymins = []
    ymaxs = []
    classes_text = []
    classes = []

    for index, row in group.object.iterrows():
        if set(['xmin_rel', 'xmax_rel', 'ymin_rel', 'ymax_rel']).issubset(
                set(row.index)):
            xmin = row['xmin_rel']
            xmax = row['xmax_rel']
            ymin = row['ymin_rel']
            ymax = row['ymax_rel']

        elif set(['xmin', 'xmax', 'ymin', 'ymax']).issubset(set(row.index)):
            xmin = row['xmin'] / width
            xmax = row['xmax'] / width
            ymin = row['ymin'] / height
            ymax = row['ymax'] / height

        xmins.append(xmin)
        xmaxs.append(xmax)
        ymins.append(ymin)
        ymaxs.append(ymax)
        classes_text.append(row['class'].encode('utf8'))
        classes.append(class_dict[row['class']])

    tf_example = tf.train.Example(
        features=tf.train.Features(
            feature={
                'image/height':
                dataset_util.int64_feature(height),
                'image/width':
                dataset_util.int64_feature(width),
                'image/filename':
                dataset_util.bytes_feature(filename),
                'image/source_id':
                dataset_util.bytes_feature(filename),
                'image/encoded':
                dataset_util.bytes_feature(encoded_jpg),
                'image/format':
                dataset_util.bytes_feature(image_format),
                'image/object/bbox/xmin':
                dataset_util.float_list_feature(xmins),
                'image/object/bbox/xmax':
                dataset_util.float_list_feature(xmaxs),
                'image/object/bbox/ymin':
                dataset_util.float_list_feature(ymins),
                'image/object/bbox/ymax':
                dataset_util.float_list_feature(ymaxs),
                'image/object/class/text':
                dataset_util.bytes_list_feature(classes_text),
                'image/object/class/label':
                dataset_util.int64_list_feature(classes),
            }))
    return tf_example


def class_dict_from_pbtxt(pbtxt_path):
    # open file, strip \n, trim lines and keep only
    # lines beginning with id or display_name
    data = [
        l.rstrip('\n').strip()
        for l in open(pbtxt_path, 'r', encoding='utf-8-sig')
        if 'id:' in l or 'display_name:'
    ]
    ids = [int(l.replace('id:', '')) for l in data if l.startswith('id')]
    names = [
        l.replace('display_name:', '').replace('"', '').strip() for l in data
        if l.startswith('display_name')
    ]

    print(data)

    # join ids and display_names into a single dictionary
    class_dict = {}
    for i in range(len(ids)):
        class_dict[names[i]] = ids[i]

    return class_dict


if __name__ == '__main__':
    parser = argparse.ArgumentParser(
        description=
        'Create a TFRecord file for use with the TensorFlow Object Detection API.',
        formatter_class=argparse.RawDescriptionHelpFormatter)
    parser.add_argument(
        'csv_input',
        metavar='csv_input',
        type=str,
        help='Path to the CSV input')
    parser.add_argument(
        'pbtxt_input',
        metavar='pbtxt_input',
        type=str,
        help='Path to a pbtxt file containing class ids and display names')
    parser.add_argument(
        'image_dir',
        metavar='image_dir',
        type=str,
        help='Path to the directory containing all images')
    parser.add_argument(
        'output_path',
        metavar='output_path',
        type=str,
        help='Path to output TFRecord')

    args = parser.parse_args()

    class_dict = class_dict_from_pbtxt(args.pbtxt_input)

    writer = tf.python_io.TFRecordWriter(args.output_path)
    path = os.path.join(args.image_dir)
    examples = pd.read_csv(args.csv_input)
    grouped = __split(examples, 'filename')

    for group in tqdm(grouped, desc='groups'):
        tf_example = create_tf_example(group, path, class_dict)
        writer.write(tf_example.SerializeToString())

    writer.close()
    output_path = os.path.join(os.getcwd(), args.output_path)
    print('Successfully created the TFRecords: {}'.format(output_path))